# Import necessary libraries

In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load data

In [ ]:
def load_data():
    movies = pd.read_csv('movies.dat', delimiter='::', engine='python', names=['movieId', 'title', 'genres'])
    ratings = pd.read_csv('ratings.dat', delimiter='::', engine='python', names=['userId', 'movieId', 'rating', 'timestamp'])
    ratings.drop('timestamp', axis=1, inplace=True)
    return movies, ratings

# Preprocess data

In [ ]:
def preprocess_data(movies):
    movies['genres'] = movies['genres'].apply(lambda x: x.replace('|', ' ').lower())
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(movies['genres'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

# Train collaborative filtering model

In [ ]:
def train_collaborative_filtering(ratings):
    reader = Reader(rating_scale=(0.5, 5.0))
    data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
    trainset = data.build_full_trainset()
    model = SVD()
    model.fit(trainset)
    return model

# Content-based recommendation function

In [ ]:
def get_content_based_recommendations(movie_title, cosine_sim, movies):
    idx = movies[movies['title'] == movie_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return movies[['movieId', 'title']].iloc[movie_indices]

# Hybrid recommendation function

In [ ]:
def hybrid_recommendation(user_id, movie_title, model, cosine_sim, movies, ratings):
    cf_predictions = []
    for movieId in movies['movieId'].unique():
        pred = model.predict(user_id, movieId).est
        cf_predictions.append((movieId, pred))
    
    cf_predictions = sorted(cf_predictions, key=lambda x: x[1], reverse=True)[:10]
    
    content_recommendations = get_content_based_recommendations(movie_title, cosine_sim, movies)
    
    movie_ids_cf = [movie[0] for movie in cf_predictions]
    movie_ids_content = content_recommendations['movieId'].values
    hybrid_movie_ids = list(set(movie_ids_cf).union(set(movie_ids_content)))
    
    return movies[movies['movieId'].isin(hybrid_movie_ids)].head(10)

# Load data, preprocess, and train the model

In [ ]:
movies, ratings = load_data()
cosine_sim = preprocess_data(movies)
model = train_collaborative_filtering(ratings)

# User input and recommendation logic

In [ ]:
# Input: User ID and movie title
user_id = int(input("Enter User ID: "))
movie_title = input("Enter a movie you like: ")

# Validate user inputs
if user_id not in ratings['userId'].unique():
    print(f"User ID {user_id} not found in the dataset.")
else:
    if movie_title not in movies['title'].values:
        print(f"Movie '{movie_title}' not found in the dataset.")
    else:
        # Generate recommendations
        recommendations = hybrid_recommendation(user_id, movie_title, model, cosine_sim, movies, ratings)

        # Display recommendations
        print(f"\nTop recommended movies for User {user_id} based on '{movie_title}':")
        for idx, row in recommendations.iterrows():
            print(f"- {row['title']}")